Ce notebook permet de créer un corpus pour le fine-tunage d'un modèle pour la summarization. Il est créé à partir des corpus suivants : 

- des plots détaillés de Wikipédia : WikiPlots : https://github.com/markriedl/WikiPlots
- des synopsis courts de IMDB : Movie Synopsis Dataset : https://www.kaggle.com/datasets/linggarmaretva/movie-synopsis-dataset

A chaque plot détaillé, on lui associe son synopsis court IMDB en considérant que c'est son résumé. 

# Importation des modules

In [41]:
import re
import csv

# Lecture des fichiers

In [25]:
# Fichiers avec les données
imdb_synopsis_file = "../Data/movie_synopsis.csv"
wiki_plots_file = "../Data/plots/plots"
wiki_plots_titles_file = "../Data/plots/titles"

# Lecture du fichier avec les synopsis IMDB
with open(imdb_synopsis_file, "r", encoding="utf-8") as csvfile:
    csv_reader = csv.DictReader(csvfile)
    
    # Création d'un dictionnaire {title: synopsis_IMDB}
    imdb_data = {row["title"].lower(): row["synopsis"] for row in csv_reader}
    
# Lecture du fichier avec les plots détaillés Wikipédia
with open(wiki_plots_file, "r", encoding="utf-8") as f:
    l = f.readline()
    
    plots = []
    temp_plot = [] # Un plot peut s'étaler sur plusieurs lignes
    
    while l:
        if l == "<EOS>\n": # Fin du plot
            plots.append("".join(temp_plot)) # Transformation de la liste en string
            temp_plot = [] # On vide la liste tampong
        else:
            temp_plot.append(l.strip())
    
        l = f.readline()
        
# Lecture du fichier avec les titres correspondants aux plots de Wikipédia
with open(wiki_plots_titles_file, "r", encoding="utf-8") as f:
    wiki_titles = []
    l = f.readline()
    while l :
        title = re.sub(r"\((\w+|\s|\d+)+\)", "", l) # Suppression des infos entre parenthèses (ex : "(movie)", "(1998)", ...)
        wiki_titles.append(title.lower().strip()) # Tout en minuscules et suppression des trailing spaces
        l = f.readline()

# Création du dataset

In [43]:
# Liste de dictionnaires : un dico = une oeuvre = {"document": "long plot wikipédia", "summary": "court synopsis IMDB"}
oeuvres = [{"document": plot, "summary": imdb_data.get(title, "")} for plot, title in zip(plots, wiki_titles)]

# Suppression des oeuvres pour lesquelles on a pas le synopsis IMDB
oeuvres_no_empty = [oeuvre for oeuvre in oeuvres if oeuvre["summary"]!=""]

# Ecriture du dataset dans un CSV
columns = oeuvres_no_empty[0].keys() # Nom des colonnes ("document", "summary")
with open('../Data/imdb_wiki_corpus.csv', 'w', encoding="utf-8", newline='') as output_file:
    dict_writer = csv.DictWriter(output_file, columns)
    dict_writer.writeheader()
    dict_writer.writerows(oeuvres_no_empty)